# LangChain Expression Language

In [ ]:
%pip install langchain langchain_openai langchain-community --upgrade

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [6]:
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

------------------------------------------

## Branching Out with _Conditional Logic:_
The runnable is initialized with a list of (condition, runnable) pairs and a default branch.

When operating on an input, the first condition that evaluates to True is selected, and the corresponding runnable is run on the input.

If no condition evaluates to True, the default branch is run on the input.

In [8]:
branch = RunnableBranch(
                (lambda x: x == 'hello', lambda x: x),
                (lambda x: isinstance(x, str), lambda x: x.upper()),
                (lambda x: "This is the default case, in case no above lambda functions match."),
            )

print(branch.invoke("hello")) # "hello"
print(branch.invoke(None)) # "This is the default case"

hello
This is the default case, in case no above lambda functions match.


---

## Branching and _Merging_

```
     Input
      / \
     /   \
 Branch1 Branch2
     \   /
      \ /
    Combine

```

### First Example:

In [3]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [4]:
chain.invoke({"input": "scrum"})

"Overall, while Scrum has many advantages, it is important to consider the potential drawbacks and assess whether they may impact the specific needs and requirements of a project or organization.\n\nThe lack of predictability in Scrum can be mitigated by implementing proper project tracking and communication mechanisms, such as regular updates and transparent reporting. This can help stakeholders and clients stay informed about the project's progress and manage expectations effectively.\n\nWhile Scrum may place less emphasis on documentation, it is still essential to maintain clear and concise documentation to ensure knowledge transfer and project maintainability. Agile practices like user stories and acceptance criteria can help capture important project details and requirements.\n\nThe dependency on team dynamics can be addressed through effective team building and collaboration strategies. Regular team-building exercises, open communication channels, and conflict resolution mechanis

### Example Two:

In [5]:
joke_chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"joke": RunnablePassthrough(), "topic": RunnablePassthrough()}
)

explain_joke = (
    ChatPromptTemplate.from_template("Explain the joke: {joke}")
    | ChatOpenAI()
    | StrOutputParser()
)

benefits_of_joke = (
    ChatPromptTemplate.from_template("List the benefits of this joke: {joke}")
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [   
            ("system", "You are responsible for generating a small analysis of a joke. The topic will be: {topic}"),
            ("ai", "{joke}. The benefits of this joke are: {benefits}"),
            ("human", "The explanation of the joke is: {explanation}"),
            ("human", "Generate a small analysis of the joke. Analysis: "),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_chain = (
    {"topic": RunnablePassthrough()}
    | joke_chain
    | {
        "explanation": explain_joke,
        "benefits": benefits_of_joke,
        "joke": itemgetter("joke"),
        "topic": itemgetter("topic"),
    }
    | final_responder
)

final_chain.invoke({"topic": "bears"})

'The joke "Why don\'t bears wear shoes? Because they have bear feet!" is a lighthearted play on words that relies on a pun to create humor. The analysis of this joke can be broken down as follows:\n\n1. Wordplay: The joke cleverly uses the homophonic words "bear" and "bare" to create a pun. By substituting "bear" for "bare," the joke suggests that bears do not wear shoes because their natural paws, referred to as "bear feet," are already suitable for walking.\n\n2. Unexpected Twist: The humor in this joke lies in the unexpected twist of words. Initially, the setup leads the listener to expect a straightforward answer to the question. However, the punchline introduces the pun, catching the listener off guard and eliciting a humorous response.\n\n3. Visual Imagery: The joke evokes a mental image of bears walking around without shoes, emphasizing the absurdity of the idea. This visualization adds to the comedic effect by creating a playful and whimsical scenario.\n\n4. Simplicity: The jok